In [10]:
import numpy as np
import hashlib
import random
import string
import hmac
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import plotly.graph_objects as go
import plotly.express as px

e = 2**52
salt = "0000000000000000000fa3b65e43e4240d71762a5bf397d5304b2596d116859c"

In [2]:
def get_result(game_hash):
    hm = hmac.new(str.encode(game_hash), b'', hashlib.sha256)
    hm.update(salt.encode("utf-8"))
    h = hm.hexdigest()
    if (int(h, 16) % 33 == 0):
        return 1
    h = int(h[:13], 16)
    e = 2**52
    return (((100 * e - h) / (e-h)) // 1) / 100.0

def get_prev_game(hash_code):
    m = hashlib.sha256()
    m.update(hash_code.encode("utf-8"))
    return m.hexdigest()

def get_all_multipliers():
    game_hash = '312608041e7dc36c53f993f49b5b2dc42d7dd348a4f5af3581b84147cb998bbd' # Update to latest game's hash for more results
    first_game = "77b271fe12fca03c618f63dfb79d4105726ba9d4a25bb3f1964e435ccf9cb209"

    results = []
    count = 0
    while game_hash != first_game:
        count += 1
        results.append(get_result(game_hash))
        game_hash = get_prev_game(game_hash)
    
    #append first game
    results.append(get_result(first_game))
    results = np.array(results)
    
    return results

#### Load data

In [3]:
multipliers = get_all_multipliers()
multipliers_chronological_order = multipliers[::-1]
multiplier_df = pd.DataFrame({'game_no': range(1,len(multipliers_chronological_order)+1), 'multiplier': multipliers_chronological_order})

In [15]:
#create lagged df 
multiplier_df

,game_no,multiplier
0,1,2.19
1,2,2.80
2,3,6.55
3,4,1.10
4,5,1.06
...,...,...
1734651,1734652,1.35
1734652,1734653,7.43
1734653,1734654,4.69
1734654,1734655,5.48


In [4]:
#train test split
train_pct = 0.8
train_df = multiplier_df.loc[:round(len(multiplier_df.index) * train_pct)]
test_df = multiplier_df.loc[round(len(multiplier_df.index) * train_pct)+1:]